# Figures 5 & 6
In this notebook, we will reproduce the panels that comprise Figures 5 and 6. Plots involving experimental data cannot be included, however, as this data is owned by the original authors and can only be distributed with their permission.

We first import the necessary packages, i.e. `NumPy`, `SciPy`, and `Matplotlib`. We also import a module called `stats_tools` which contains our own statistics functions that are specifically written for the synaptic data. This, in turn, requires `SciPy` and `Scikit-learn`.

In [2]:
import numpy as np
from scipy.stats import binned_statistic, linregress, norm
from scipy.linalg import lstsq
from scipy.optimize import curve_fit
from scipy.ndimage import uniform_filter1d
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.legend_handler import HandlerTuple
import matplotlib.colors as mcolors

from stats_tools import curate_data, slope_vs_dt, cv

We define some layout settings for the plots as a few other auxiliary variables.

In [3]:
plt.rcParams.update({'pdf.fonttype': 42,
                     'pdf.use14corefonts': False,
                     'ps.useafm': False,
                     'figure.constrained_layout.use': True,
                     'figure.dpi': 150.,
                     'font.size': 7,
                     'figure.titlesize': 7,
                     'figure.labelsize': 7,
                     'axes.labelsize': 7,
                     'axes.labelpad': 2,
                     'axes.titlesize': 7,
                     'axes.titlepad': 2,
                     'figure.titlesize': 7,
                     'figure.labelweight': 'light',
                     'figure.titleweight': 'light',
                     'axes.labelweight': 'light',
                     'axes.titleweight': 'light',
                     'font.family': 'sans-serif',
                     'font.sans-serif': 'Arial',
                     'font.weight': 'light',
                     'mathtext.fontset': 'custom',
                     'mathtext.it': 'Arial:italic',
                     'mathtext.bf': 'Arial:bold',
                     'mathtext.sf': 'Arial:light',
                     'mathtext.default': 'sf',
                     'axes.spines.right': False,
                     'axes.spines.top': False,
                     'axes.linewidth': 0.5,
                     'xtick.major.size': 2.5,
                     'xtick.minor.size': 1.5,
                     'ytick.major.size': 2.5,
                     'ytick.minor.size': 1.5,
                     'xtick.major.width': 0.5,
                     'xtick.minor.width': 0.5,
                     'ytick.major.width': 0.5,
                     'ytick.minor.width': 0.5,
                     'xtick.major.pad': 3.5/3,
                     'xtick.minor.pad': 3.5/3,
                     'ytick.major.pad': 3.5/3,
                     'ytick.minor.pad': 3.5/3,
                     'lines.linewidth': 1.5,
                     'lines.markersize': 4.,
                     'hatch.color': 'w',
                     'hatch.linewidth': 0.5,})

mm = 1/2.54/10  # mm in inches
cm = 1/2.54  # cm in inches

Load data files containing the results of the simulations. Synaptic data files are stored typically as `NumPy` binaries (`.npy`).

In [4]:
sim_dict = np.load('./data/diffusion_k00_u-hom_tau-30.npy', allow_pickle=True).item()

We now run the statistical analysis on the simulated synaptic population.

In [9]:
# the simulated time step represents 10 min of real time
dt = 10/60

# we select 144 steps of simulated data
n_steps = 144

# loop over datasets with different z-values
for z in [1, 2, 3, 4, 5]:
    print(f'z: {z}')
    if sim_dict['num'][z][-1] < 0.1:
        continue
    sim_clean = curate_data(sim_dict['w'][z][:,-n_steps:], dt=dt)
    sim_clean = cv(sim_clean, n_resamples=1000, n_steps=n_steps, seed=123)
    # sim_clean = slope_vs_dt(sim_clean, n_resamples=100, seed=boot_seed, slope='conv')
    sim_dict['z'+str(z)] = sim_clean
print('FINISHED!')

z: 1
Curation... Done!
CVs... Done!
z: 2
Curation... Done!
CVs... Done!
z: 3
Curation... Done!
CVs... Done!
z: 4
Curation... Done!
CVs... Done!
z: 5
Curation... Done!
CVs... Done!
Done!


In [ ]:
# save cleaned and analyzed dataset
#np.save('./data/diffusion_k00_u-hom_tau-30_clean_extra-CV.npy', sim_dict)